In [7]:
import sqlite3
import requests
import datetime
import pandas as pd 

In [1]:
ACCESS_TOKEN = os.environ.get('ACCESS_TOKEN')

In [2]:
teams = {"Dodgers" : {"teamID": 1061, "venueID": 744},
         "Angels": {"teamID": 1062, "venueID" : 9763},
         "Giants" : {"teamID" : 197, "venueID" : 82}}

In [3]:
for team in teams:
    print(teams[team]["teamID"])

1061
1062
197


In [22]:
def get_ticket_info(teamID, venueID):
    url = f"https://api.stubhub.com/sellers/search/events/v3?venueId={venueID}&performerId={teamID}&parking=false&sort=eventDateLocal%20asc&rows=81"
    headers = { 
    "Authorization": "Bearer " + ACCESS_TOKEN,
    "Accept": "application/json"
    }
    r = requests.get(url, headers=headers)
    return r

def create_df(r):
    events = r.json()['events']
    prices = {event['id']:{
         "price": event['ticketInfo']['minPrice'],
         "time" : event['eventDateLocal'],
         "name" : event['name'],
         "extractTime":datetime.datetime.today(),
         "homeTeam" : event['performers'][0]['name'],
         "awayTeam" : event['performers'][1]['name'],
         "ticket_count": event["ticketInfo"]["totalTickets"]} for event in events}
    df = pd.DataFrame(prices).T.reset_index().rename({"index":"id"}, axis = 1)
    df["unique_name"] = df["name"] + ": " + df["time"].str[0:10]
    df['time'] = pd.to_datetime(df['time'].str[0:10])
    df['days_to_game'] = (df['time'] - df['extractTime'])
    df['days_to_game'] = df['days_to_game'].apply(lambda X: X.days)
    return df

In [11]:
for team in teams:
    teamID = teams[team]["teamID"]
    venueID = teams[team]["venueID"]
    r = get_ticket_info(teamID, venueID)
    df = create_df(r)
    with sqlite3.connect("../tickets.db") as conn: 
        df.to_sql("extraction", conn, if_exists = "append", index = False)

In [2]:
with sqlite3.connect("tickets.db") as conn: 
    cursor = conn.cursor()
    cursor.execute("SELECT sql FROM sqlite_master WHERE type='table';")

In [61]:
for result in cursor.fetchall():
    print(result[0])

CREATE TABLE "extraction" (
"price" REAL,
  "time" TEXT,
  "name" TEXT,
  "extractTime" TIMESTAMP
)


In [8]:
with sqlite3.connect("../tickets.db") as conn: 
    cmd = "SELECT * FROM extraction"
    df2 = pd.read_sql_query(cmd, conn)

In [9]:
df2

,id,price,time,name,extractTime,homeTeam,awayTeam,days_to_game
0,104722879,58.62,2021-06-11 00:00:00,Texas Rangers at Los Angeles Dodgers,2021-06-08 17:51:48.284897,Los Angeles Dodgers,Texas Rangers,2
1,104725408,29.16,2021-06-12 00:00:00,Texas Rangers at Los Angeles Dodgers,2021-06-08 17:51:48.284897,Los Angeles Dodgers,Texas Rangers,3
2,104725409,28.04,2021-06-13 00:00:00,Texas Rangers at Los Angeles Dodgers,2021-06-08 17:51:48.284897,Los Angeles Dodgers,Texas Rangers,4
3,104724531,35.55,2021-06-14 00:00:00,Philadelphia Phillies at Los Angeles Dodgers,2021-06-08 17:51:48.284897,Los Angeles Dodgers,Philadelphia Phillies,5
4,104725410,24.99,2021-06-15 00:00:00,Philadelphia Phillies at Los Angeles Dodgers,2021-06-08 17:51:48.284897,Los Angeles Dodgers,Philadelphia Phillies,6
...,...,...,...,...,...,...,...,...
611,104738380,17.17,2021-09-30 00:00:00,Arizona Diamondbacks at San Francisco Giants,2021-06-10 12:10:34.477740,San Francisco Giants,Arizona Diamondbacks,111
612,104736599,17.17,2021-10-01 00:00:00,Arizona Diamondbacks at San Francisco Giants,2021-06-10 12:10:34.477742,San Francisco Giants,Arizona Diamondbacks,112
613,104736601,46.19,2021-10-02 00:00:00,San Diego Padres at San Francisco Giants,2021-06-10 12:10:34.477744,San Francisco Giants,San Diego Padres,113
614,104738382,58.47,2021-10-02 00:00:00,San Diego Padres at San Francisco Giants,2021-06-10 12:10:34.477746,San Francisco Giants,San Diego Padres,113


In [16]:
df2["unique_name"] = df2["name"] + ": " + df2["time"].str[0:10]
df2

,id,price,time,name,extractTime,homeTeam,awayTeam,days_to_game,unique_name
0,104722879,58.62,2021-06-11 00:00:00,Texas Rangers at Los Angeles Dodgers,2021-06-08 17:51:48.284897,Los Angeles Dodgers,Texas Rangers,2,Texas Rangers at Los Angeles Dodgers: 2021-06-11
1,104725408,29.16,2021-06-12 00:00:00,Texas Rangers at Los Angeles Dodgers,2021-06-08 17:51:48.284897,Los Angeles Dodgers,Texas Rangers,3,Texas Rangers at Los Angeles Dodgers: 2021-06-12
2,104725409,28.04,2021-06-13 00:00:00,Texas Rangers at Los Angeles Dodgers,2021-06-08 17:51:48.284897,Los Angeles Dodgers,Texas Rangers,4,Texas Rangers at Los Angeles Dodgers: 2021-06-13
3,104724531,35.55,2021-06-14 00:00:00,Philadelphia Phillies at Los Angeles Dodgers,2021-06-08 17:51:48.284897,Los Angeles Dodgers,Philadelphia Phillies,5,Philadelphia Phillies at Los Angeles Dodgers: ...
4,104725410,24.99,2021-06-15 00:00:00,Philadelphia Phillies at Los Angeles Dodgers,2021-06-08 17:51:48.284897,Los Angeles Dodgers,Philadelphia Phillies,6,Philadelphia Phillies at Los Angeles Dodgers: ...
...,...,...,...,...,...,...,...,...,...
611,104738380,17.17,2021-09-30 00:00:00,Arizona Diamondbacks at San Francisco Giants,2021-06-10 12:10:34.477740,San Francisco Giants,Arizona Diamondbacks,111,Arizona Diamondbacks at San Francisco Giants: ...
612,104736599,17.17,2021-10-01 00:00:00,Arizona Diamondbacks at San Francisco Giants,2021-06-10 12:10:34.477742,San Francisco Giants,Arizona Diamondbacks,112,Arizona Diamondbacks at San Francisco Giants: ...
613,104736601,46.19,2021-10-02 00:00:00,San Diego Padres at San Francisco Giants,2021-06-10 12:10:34.477744,San Francisco Giants,San Diego Padres,113,San Diego Padres at San Francisco Giants: 2021...
614,104738382,58.47,2021-10-02 00:00:00,San Diego Padres at San Francisco Giants,2021-06-10 12:10:34.477746,San Francisco Giants,San Diego Padres,113,San Diego Padres at San Francisco Giants: 2021...


In [10]:
df2.pivot_table(values = "price", columns = "days_to_game", index = "name")

days_to_game,-2,-1,0,1,2,3,4,5,6,7,...,107,108,109,110,111,112,113,114,115,116
name,,,,,,,,,,,,,,,,,,,,,
Arizona Diamondbacks at Los Angeles Dodgers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arizona Diamondbacks at San Francisco Giants,NaN,NaN,NaN,NaN,NaN,26.550,24.333333,23.97,27.984,22.870000,...,NaN,NaN,29.01,26.246667,21.0175,17.17,17.170000,NaN,NaN,NaN
Atlanta Braves at Los Angeles Dodgers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Atlanta Braves at San Francisco Giants,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Baltimore Orioles at Los Angeles Angels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Boston Red Sox at Los Angeles Angels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chicago Cubs at Los Angeles Dodgers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Colorado Rockies at Los Angeles Angels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Colorado Rockies at Los Angeles Dodgers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
df['price'] = df['price'].astype("float32")

In [116]:
conn.close()

In [7]:
df2

,id,price,time,name,extractTime,days_to_game
0,104722879,58.62,2021-06-11 00:00:00,Texas Rangers at Los Angeles Dodgers,2021-06-08 21:51:33.574579,2
1,104725408,31.46,2021-06-12 00:00:00,Texas Rangers at Los Angeles Dodgers,2021-06-08 21:51:33.574589,3
2,104725409,28.04,2021-06-13 00:00:00,Texas Rangers at Los Angeles Dodgers,2021-06-08 21:51:33.574593,4
3,104724531,35.55,2021-06-14 00:00:00,Philadelphia Phillies at Los Angeles Dodgers,2021-06-08 21:51:33.574595,5
4,104725410,25.32,2021-06-15 00:00:00,Philadelphia Phillies at Los Angeles Dodgers,2021-06-08 21:51:33.574597,6
...,...,...,...,...,...,...
99,104724625,36.37,2021-09-29 00:00:00,San Diego Padres at Los Angeles Dodgers,2021-06-08 22:15:26.493996,112
100,104724627,35.14,2021-09-30 00:00:00,San Diego Padres at Los Angeles Dodgers,2021-06-08 22:15:26.493997,113
101,104724629,35.14,2021-10-01 00:00:00,Milwaukee Brewers at Los Angeles Dodgers,2021-06-08 22:15:26.493999,114
102,104724631,44.96,2021-10-02 00:00:00,Milwaukee Brewers at Los Angeles Dodgers,2021-06-08 22:15:26.494000,115


In [23]:
r = get_ticket_info(teams["Dodgers"]["teamID"], teams["Dodgers"]["venueID"])
create_df(r)

,id,price,time,name,extractTime,homeTeam,awayTeam,ticket_count,unique_name,days_to_game
0,104722879,100.2,2021-06-11,Texas Rangers at Los Angeles Dodgers,2021-06-10 17:22:01.700180,Los Angeles Dodgers,Texas Rangers,622,Texas Rangers at Los Angeles Dodgers: 2021-06-11,0
1,104725408,37.65,2021-06-12,Texas Rangers at Los Angeles Dodgers,2021-06-10 17:22:01.700180,Los Angeles Dodgers,Texas Rangers,1070,Texas Rangers at Los Angeles Dodgers: 2021-06-12,1
2,104725409,24.45,2021-06-13,Texas Rangers at Los Angeles Dodgers,2021-06-10 17:22:01.700180,Los Angeles Dodgers,Texas Rangers,1464,Texas Rangers at Los Angeles Dodgers: 2021-06-13,2
3,104724531,28.1,2021-06-14,Philadelphia Phillies at Los Angeles Dodgers,2021-06-10 17:22:01.700180,Los Angeles Dodgers,Philadelphia Phillies,1264,Philadelphia Phillies at Los Angeles Dodgers: ...,3
4,104725410,24.99,2021-06-15,Philadelphia Phillies at Los Angeles Dodgers,2021-06-10 17:22:01.700180,Los Angeles Dodgers,Philadelphia Phillies,3467,Philadelphia Phillies at Los Angeles Dodgers: ...,4
5,104725411,35.95,2021-06-16,Philadelphia Phillies at Los Angeles Dodgers,2021-06-10 17:22:01.700180,Los Angeles Dodgers,Philadelphia Phillies,3326,Philadelphia Phillies at Los Angeles Dodgers: ...,5
6,104722883,47.42,2021-06-24,Chicago Cubs at Los Angeles Dodgers,2021-06-10 17:22:01.700180,Los Angeles Dodgers,Chicago Cubs,3579,Chicago Cubs at Los Angeles Dodgers: 2021-06-24,13
7,104722884,46.19,2021-06-25,Chicago Cubs at Los Angeles Dodgers,2021-06-10 17:22:01.700180,Los Angeles Dodgers,Chicago Cubs,4268,Chicago Cubs at Los Angeles Dodgers: 2021-06-25,14
8,104722885,49.3,2021-06-26,Chicago Cubs at Los Angeles Dodgers,2021-06-10 17:22:01.700180,Los Angeles Dodgers,Chicago Cubs,3601,Chicago Cubs at Los Angeles Dodgers: 2021-06-26,15
9,104722886,35.22,2021-06-27,Chicago Cubs at Los Angeles Dodgers,2021-06-10 17:22:01.700180,Los Angeles Dodgers,Chicago Cubs,4295,Chicago Cubs at Los Angeles Dodgers: 2021-06-27,16
